### **CSV Automation**

#### **Introduction**

Data import plays a pivotal role in data science. Knowing how to do it prevents the possibility of failure of the model. To achieve this, various methods, which differ depends on the data types (e.g. .csv, .txt, .json), are possible.

Data acquisition is a large part of many data analytics projects and system development life cycles. This article will show you how to write a simple Python program that uses the BULK INSERT utility to rapidly insert data from a CSV file into a SQL Server database table.

#### **Import the Necessary Libraries** 

In [ ]:
#import libraries

import os
import pandas as pd
import numpy as np
import psycopg2

#### **Find and move the CSV files to new directory from working directory**

In [ ]:
#find csv files in the working directory

csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [ ]:
#make new directory

dataset_dir = 'datasets'

#create the bash command to make a new directory
# mkdir dataset_dir

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [ ]:
#move the CV files in the new directory

#mv filename directory

for csv in csv_files:
    mv_file ="mv '{0}' {1}".format(csv, dataset_dir)
    os.system(mv_file)
    print(mv_file)


#### **create Pandas DataFrame for CSV**

In [ ]:
#Create Pandas dataframe for csv to clean the data

data_path = os.getcwd()+'/'+dataset_dir+'/'

df={}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
        

#### **Clean the table,columns name and Maps the Dtype of CSV file to Upload into the DataBase**

In [ ]:
# Clean the column, table name and maps the pandas datatype to sql datatype
# Then upload the CSV file into the database

for k in csv_files:
    
    dataframe = df[k]
    
    #clean table name
    clean_tbl_name = k.lower().replace(" ","_").replace("?","")\
                      .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","")\
                      .replace(")","").replace(r"(","").replace("$","")
    
    
    #remove .csv extension from clean_tbl_name
    table_name='{0}'.format(clean_tbl_name.split('.')[0])
    print(table_name)
    
    #clean columns name
    dataframe.columns = [x.lower().replace(" ","_").replace("?","")\
                          .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","")\
                          .replace(")","").replace(r"(","").replace("$","") for x in dataframe.columns]
    
    #replacement dictionary that maps pandas dtypes to sql dtypes
    replacements = {
        'object':'varchar',
        'float64':'float',
        'int64':'int',
        'datetime64':'timestamp',
        'timedelta64[ns]':'varchar'
    }
    
    #table schema
    col_str=",".join("{} {}".format(n,d) for (n,d) in zip(dataframe.columns,dataframe.dtypes.replace(replacements)))
    print(col_str)
    
    #open database connection
    host = 'project-db.******.ap-south-1.rds.amazonaws.com'
    port = '5432'
    dbname = 'postgres'
    user = '*****'
    password = '*******'
    
    #connect to database
    conn_string = "host=%s port=%s database=%s user=%s password=%s" % (host, port, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    cursor=conn.cursor()
    print('Database opened succesfully')
    
    #drop table if exixsts
    cursor.execute("drop table if exists %s") % (table_name)
    
    #create table
    cursor.execute("create table %s (%s)") % (table_name, col_str)
    print('{0} was created succesfully'.format(table_name))
    
    #insert values into the table
    #save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')
    
    #open csv file into object
    my_file = open(k)
    print ('file opened in memory')
    
    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """
    
    cursor.copy_expert(sql=SQL_STATEMENT % table_name, file=my_file)
    print ('file copied to db')
    
    cursor.execute("grant select on table %s to public" % table_name)
    conn.commit()
    cursor.close()
    print('table {0} imported to db completed'.format(table_name))
    
#for loop and message
print ('all tables have been successfully imported into the db')



